In [1]:
import tensorflow as tf

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pretrained_alexnet import AlexNet
#import urllib
#import cStringIO
#from PIL import Image
#from HTMLParser import HTMLParser
#from scipy.misc import imread, imsave, imresize


#%matplotlib inline
#plt.style.use('ggplot')

Falling back to protocol buffer implementation.
* Conversions will be drastically slower.
* This backend is UNTESTED!


In [2]:
aesthetic_ratings = pd.read_table("AVA.txt", header=None, delimiter=" ")
image_data = np.load("test2.npy")

In [3]:
scores = {}
rows, cols = aesthetic_ratings.shape
weighted_avgs = {}
for i in xrange(rows):
    row = aesthetic_ratings.iloc[i].tolist()
    ratings = row[2:12]
    numbers = sum([ ratings[k] * (k + 1) for k in xrange(len(ratings)) ])
    num_votes = sum(ratings)
    weighted_avgs[row[1]] = numbers / float(num_votes)

In [4]:
image_data_dict = image_data.item()

flattened_images = []
flattened_score = []
img_paths = []
for key in image_data_dict.keys():
    if key not in weighted_avgs:
        continue
    flattened_images.append(image_data_dict[key].flatten())
    flattened_score.append(weighted_avgs[key])
    img_paths.append(str(key) + ".png")

In [5]:
distributed_scores = []
for score in flattened_score:
    if (score < 2):
        distributed_scores.append(np.array([1,0,0,0,0]))
    elif (score >= 2 and score < 4):
        distributed_scores.append(np.array([0,1,0,0,0]))
    elif (score >= 4 and score < 6):
        distributed_scores.append(np.array([0,0,1,0,0]))
    elif (score >= 6 and score < 8):
        distributed_scores.append(np.array([0,0,0,1,0]))
    else:
        distributed_scores.append(np.array([0,0,0,0,1]))

In [6]:
print np.array(flattened_images).shape
print np.array(flattened_score).shape
#cnn_input = (np.array(flattened_images).reshape(2479,1), np.array(distributed_scores).reshape(2479,5))
cnn_input = (np.array(flattened_images).reshape(2479,1), np.zeros((2479, 5)))

print cnn_input[0].shape
#np.savez("train_data", cnn_input)

(2479,)
(2479,)
(2479, 1)


In [7]:
class AnimalData(object):
    def __init__(self, data, image_paths, dirname):
        self.image_paths = image_paths
        self.data = data
        self.dirname = dirname
        self.mean = np.array([104., 117., 124.])
        
    def read_image(self, filename):
        img = cv2.imread(os.path.join(self.dirname, filename))
        h, w, c = np.shape(img)       
        img = img.astype(np.float32)
        crop_size = 227
        img -= self.mean
        h, w, c = img.shape
        ho, wo = ((h-crop_size)/2, (w-crop_size)/2)
        img = img[ho:ho+crop_size, wo:wo+crop_size, :]
        img = img[None, ...]
        return img

    def batches(self, n):
        for i in xrange(0, 2497, n):
            images = np.concatenate(map(self.read_image, self.image_paths[i:i+n]), axis=0)
            labels = self.data[1][i:i + n]
            yield (images, labels)

In [8]:
test_data = tf.placeholder(tf.float32, shape=(37, 227, 227, 3))
test_labels = tf.placeholder(tf.int32, shape=(37, 5))
animaldata = AnimalData(cnn_input, img_paths, "animal_images")

In [9]:
net = AlexNet({'data': test_data})

In [10]:
probs = net.get_output()
#top_k_op = tf.nn.in_top_k(probs, test_labels, 5)
print probs

Tensor("prob:0", shape=TensorShape([Dimension(37), Dimension(5)]), dtype=float32)


In [11]:
cnn_input[0].reshape(2479, 1)
print cnn_input[0].shape
print cnn_input[1].shape
a = np.zeros((2479, 1000))
print a.shape

(2479, 1)
(2479, 5)
(2479, 1000)


In [12]:
with tf.Session() as sesh:
    net.load('pretrained_weights_alexnet.npy', sesh)
    for idx, (images, labels) in enumerate(animaldata.batches(37)):
        #sess.run(train_op, feed_dict={x:batch[0][i-37:i,:], y_: batch[1][i-37:i,:]})
        print sesh.run(probs, feed_dict={test_data:images, test_labels:labels}).shape
    #    correct += np.sum(sesh.run(top_k_op, feed_dict={test_data:images, test_labels:labels}))
    #    count += len(images)
    #    cur_accuracy = float(correct)*100/count
    #    print('{:>6}/{:<6} {:>6.2f}%'.format(count, total, cur_accuracy))
#print('Top %s Accuracy: %s'%(top_k, float(correct)/total))

ValueError: Dimensions Dimension(5) and Dimension(1000) are not compatible